In [6]:
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import BatchNormalization
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from keras.utils import np_utils


CNNLable = ['S10','H3','S9','SJ','HA','SA','CJ','D3','C5','S5','DJ','H10','C2','S10','HJ','C4','SK','CK','C6','SK','D6','D5','C3','C10','H7','D10','C9','CQ','S7','SA','C6','H6','H10',
       'D2','DQ','D4','H4','D4','S8','CA','DK','CQ','CK','S3','D7','C2','H3','HJ','CJ','S4','DQ','C4','DA','D8','C7','D2','D9','C3','HA','HK','SQ','H4',
       'S2','C8','C7','H2','HQ','HQ','D3','S6','S3','SJ','H8','CA','D8','DJ','D9','H8','D10','H6','DA','S2','D5','H5','H7','S4','DK','H9','H2','C8',
       'C9','D6','SQ','S6','HK','S9','S8','C5','H9','D7','S5','S7','H5','C10']

def lableTransform(lableList):
    v = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
    labelValue = []
    for i in CNNLable:
        value = 0
        value = v.index(i[1:])
        if i[0] == 'H':
            value += 13
        elif i[0] == 'D':
            value += 26
        elif i[0] == 'C':
            value += 39
        labelValue.append(value)
        
    labelValue = np_utils.to_categorical(labelValue)
    return labelValue

def decode(num):
    
    if num // 13 == 0:
        numID = 'S'+str(trans(num))
    elif num // 13 == 1:
        numID = 'H'+str(trans(num))
    elif num // 13 == 2:
        numID = 'D'+str(trans(num))
    elif num // 13 == 3:
        numID = 'C'+str(trans(num))
    return numID

def trans(num):
    num += 1
    if num % 13 == 1:
        return 'A'
    elif num % 13 == 11:
        return 'J'
    elif num % 13 == 12:
        return 'Q'
    elif num % 13 == 0:
        return 'K'
    else:
        return str(num % 13)

# Load image
dirPath = r"cards"
results = [f for f in os.listdir(dirPath) if os.path.isfile(os.path.join(dirPath, f))]
images = []
for img in results:
    imgcv = cv2.imread(dirPath+'\\'+img)
    imgcv = cv2.cvtColor(imgcv, cv2.COLOR_BGR2RGB)
    images.append(imgcv)

images = np.array(images) # Transform to array


# ================ model disign ==========================

height, width, channels = 46,34,3
classifier=Sequential()

# CNN layers
classifier.add(Convolution2D(32,3,3,input_shape=(height, width, channels),activation='relu'))
classifier.add(BatchNormalization())
classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPooling2D(pool_size=(2,2)))
classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(Convolution2D(32,3,3,activation='relu'))
classifier.add(Convolution2D(32,3,3,activation='relu'))

#flatten
classifier.add(Flatten())

classifier.add(Dense(output_dim=100,activation='relu'))
classifier.add(Dropout(p=0.3))
# output
classifier.add(Dense(output_dim=52,activation='softmax'))
# compile model
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
# summary
classifier.summary()

# ==================== generate more data =======================
'''
By using keras image preprocessing to generate more data such as rotating and width, height
shifting
'''
datagen = keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# ================ training =====================
# set learning rate
lr = keras.callbacks.ReduceLROnPlateau(min_lr=0.0000001)

classifier.fit_generator(datagen.flow(x=images, y= lableTransform(CNNLable)),   
                          epochs=500, steps_per_epoch=30,callbacks= [lr]) 

# ============== evaluate model ==============

loss, accuracy = classifier.evaluate(images,labelValue)
print('test loss: ', loss)
print('test accuracy: ', accuracy)

# ============= plot figures ================
for i ,j in zip(images,np.argmax(classifier.predict(images),axis=1)):
    
    print(decode(j))
    print('Confident: ', np.max(classifier.predict(images)))
    _, ax = plt.subplots(figsize=(4, 2))
    ax.imshow(i)
    plt.show()

    

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:82: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(46, 34, 3..., activation="relu")`
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:84: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:88: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu")`
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activ

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 44, 32, 32)        896       
_________________________________________________________________
batch_normalization_3 (Batch (None, 44, 32, 32)        128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 42, 30, 32)        9248      
_________________________________________________________________
batch_normalization_4 (Batch (None, 42, 30, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 21, 15, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 19, 13, 32)        9248      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 17, 11, 32)       

C:\Users\User\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:716: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
C:\Users\User\Anaconda3\lib\site-packages\keras_preprocessing\image\image_data_generator.py:724: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '


30/30 [==============================] - 3s 107ms/step - loss: 4.0104 - accuracy: 0.0182
Epoch 2/500
 7/30 [======>.......................] - ETA: 0s - loss: 3.9006 - accuracy: 0.0300

C:\Users\User\Anaconda3\lib\site-packages\keras\callbacks\callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


30/30 [==============================] - 1s 24ms/step - loss: 3.9004 - accuracy: 0.0328
Epoch 3/500
30/30 [==============================] - 1s 26ms/step - loss: 3.7541 - accuracy: 0.0404
Epoch 4/500
30/30 [==============================] - 1s 26ms/step - loss: 3.6331 - accuracy: 0.0429
Epoch 5/500
30/30 [==============================] - 1s 26ms/step - loss: 3.4912 - accuracy: 0.0530
Epoch 6/500
30/30 [==============================] - 1s 24ms/step - loss: 3.4224 - accuracy: 0.0573
Epoch 7/500
30/30 [==============================] - 1s 27ms/step - loss: 3.3181 - accuracy: 0.0794
Epoch 8/500
30/30 [==============================] - 1s 25ms/step - loss: 3.2261 - accuracy: 0.0896
Epoch 9/500
30/30 [==============================] - 1s 27ms/step - loss: 3.1084 - accuracy: 0.1023
Epoch 10/500
30/30 [==============================] - 1s 24ms/step - loss: 3.0574 - accuracy: 0.1146
Epoch 11/500
30/30 [==============================] - 1s 26ms/step - loss: 3.0561 - accuracy: 0.1107
Epoch 12/5

KeyboardInterrupt: 